In [13]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean

from avalanche.models import IcarlNet
from avalanche.training.strategies import ICaRL
from avalanche.logging import InteractiveLogger
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [14]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [15]:
pixel_transforms = transforms.Compose([transforms.ToTensor()])
per_pixel_mean = get_dataset_per_pixel_mean(CIFAR10('../data', train=True, download=True, transform=pixel_transforms))

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - per_pixel_mean,
              icarl_cifar100_augment_data,
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - per_pixel_mean,
              ]
       ),
       None,
       )
)

train_set = CIFAR10('../data', train=True, download=True)
test_set = CIFAR10('../data', train=False, download=True)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [16]:
# splitcifar = SplitCIFAR10(n_experiences=5, seed=seed, dataset_root='../data')

# train_stream = splitcifar.train_stream
# test_stream = splitcifar.test_stream

# print(splitcifar.classes_order)

In [17]:
interactive_logger = InteractiveLogger()
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger])

In [33]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 2
lr_milestones = [49, 63]    # ?
lr_factor = 5.0 # ?
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order,
                        )

model = IcarlNet(num_classes=num_class)    # n = ResidualBlock, c = input_dim
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
sched = LRSchedulerPlugin(
        MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor)
    )

In [34]:
memory_size = 2000
train_batch = 128
eval_batch = 64
epoch = 10

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

In [37]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

for idx, experience in enumerate(train_stream):
    dataset = experience.dataset
    
    print('Train dataset contains', len(dataset), 'patterns')

    for x, y, task_label in dataset:
        # Train Step...
        pass
    
    test_experience = test_stream[idx]
    cumulative_test = test_stream[:idx+1]

Train dataset contains 25000 patterns
Train dataset contains 25000 patterns


In [36]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp, num_workers=4)
    strategies.eval(eval_exps, num_workers=4)

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


PicklingError: Can't pickle <function <lambda> at 0x000001B8AF9EAC10>: attribute lookup <lambda> on __main__ failed